<a href="https://colab.research.google.com/github/thisisindrajit/Diabetic-Retinopathy/blob/main/Diabetic_Retinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## References

- [Paper 1](https://ieeexplore.ieee.org/document/6139346)

- [Paper 2](https://iopscience.iop.org/article/10.1088/1757-899X/1070/1/012049)

- [Paper 3](https://www.sciencedirect.com/science/article/pii/S2352914820302069)
- 
[Paper 4](https://www.researchgate.net/publication/241634815_Detection_and_classification_of_diabetic_retinopathy_using_retinal_images)

- [Article](https://towardsdatascience.com/blindness-detection-diabetic-retinopathy-using-deep-learning-on-eye-retina-images-baf20fcf409e)

- [Sample code](https://github.com/debayanmitra1993-data/Blindness-Detection-Diabetic-Retinopathy-)

- [Link to DDR Dataset](https://drive.google.com/drive/folders/1z6tSFmxW_aNayUqVxx6h6bY4kwGzUTEC?usp=sharing)

# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import numpy as np
import pandas as pd
import imageio
import PIL #pillow
import cv2
import tensorflow as tf
from tensorflow import keras
import scipy
import os
import sys
import shutil
import matplotlib.pyplot as plt

# Modifying directories for easy processing

In [ ]:
dataset_dir = "/content/drive/MyDrive/Sem 7/Project/Dataset/DR_grading/"
train_dir = '/content/drive/MyDrive/Sem 7/Project/Dataset/DR_grading/train/'
test_dir = '/content/drive/MyDrive/Sem 7/Project/Dataset/DR_grading/test/'
valid_dir = '/content/drive/MyDrive/Sem 7/Project/Dataset/DR_grading/valid/'

## Modifying Training directory

In [17]:
train_csv = pd.read_csv(dataset_dir + "train.csv")

In [21]:
for i in range(len(train_csv)):
  if train_csv["output"][i] != 5: # 5 is ungradeable
    file_name = train_csv["path"][i]
    output = train_csv["output"][i]

    original = train_dir + file_name
    target = train_dir + str(output) + '/' + file_name

    #print(original, target)
    try:
      shutil.move(original,target)
    except:
      continue

In [ ]:
count = 0

for i in range(len(train_csv)):
  if train_csv["output"][i] != 5:
    count += 1

print(count)

6260


## Modifying Validation directory

In [22]:
valid_csv = pd.read_csv(dataset_dir + "valid.csv")

In [23]:
for i in range(len(valid_csv)):
  if valid_csv["output"][i] != 5: # 5 is ungradeable
    file_name = valid_csv["path"][i]
    output = valid_csv["output"][i]

    original = valid_dir + file_name
    target = valid_dir + str(output) + '/' + file_name

    #print(original, target) 
    try:
      shutil.move(original,target)
    except:
      continue

In [24]:
count = 0

for i in range(len(valid_csv)):
  if valid_csv["output"][i] != 5:
    count += 1

print(count)

2503
